In [ ]:
pip install tqdm selenium webdriver-manager beautifulsoup4 pandas

In [ ]:
import time
import urllib.request
import urllib.parse
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import pandas as pd

### 드라이버 설정

In [ ]:
# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)

# Naver API key 입력
client_id = 'k9kbYZS4KZ54NuuEitRI'
client_secret = 'sBiu_KktIH'


### 크롤링 키워드 리스트

In [ ]:
# 크롤링할 키워드 리스트
keywords = ["세탁기", "건조기"]
end = 1  # 각 키워드 당 검색할 페이지 수

### 데이터 수집

In [ ]:
# 정규표현식
pattern1 = '<[^>]*>'

# 전체 결과 저장 리스트
all_data = []

for keyword in tqdm(keywords, desc="전체 키워드 진행"):
    print(f"\n▶ 크롤링 시작: '{keyword}' 키워드")

    urls_list = []
    postdate = []
    titles = []
    contents = []

    encText = urllib.parse.quote(keyword)

    # API 호출로 URL 수집
    for start in range(end):
        url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={(start * 10) + 1}"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)

        try:
            response = urllib.request.urlopen(request)
            if response.getcode() == 200:
                data = json.loads(response.read().decode('utf-8'))['items']
                for row in data:
                    if 'blog.naver' in row['link']:
                        urls_list.append(row['link'])
                        postdate.append(row['postdate'])
                        titles.append(re.sub(pattern=pattern1, repl='', string=row['title']))
            else:
                print("API Error Code:", response.getcode())
            time.sleep(1)
        except Exception as e:
            print(f"[API 호출 실패] {e}")

    # 블로그 본문 수집
    for url in tqdm(urls_list, desc=f"[{keyword}] 블로그 본문 수집"):
        try:
            driver.get(url)
            time.sleep(2)

            iframe = driver.find_element(By.ID, "mainFrame")
            driver.switch_to.frame(iframe)

            source = driver.page_source
            html = BeautifulSoup(source, 'html.parser')

            content = html.select("div.se-main-container")
            content = ''.join(str(content))
            content = re.sub(pattern=pattern1, repl='', string=content)
            content = content.replace('\n', '').replace('\u200b', '')
            contents.append(content)
        except Exception as e:
            print(f"[본문 수집 실패] {url} → {e}")
            contents.append("error")

    # 리스트 길이 불일치 방지
    min_len = min(len(postdate), len(titles), len(contents), len(urls_list))
    if len(postdate) != len(contents):
        print(f"[경고] 리스트 길이 불일치: date={len(postdate)}, title={len(titles)}, content={len(contents)}, url={len(urls_list)}")
    
    keyword_df = pd.DataFrame({
        'keyword': [keyword] * min_len,
        'date': postdate[:min_len],
        'title': titles[:min_len],
        'content': contents[:min_len],
        'url': urls_list[:min_len]
    })

    all_data.append(keyword_df)

## 바로 아래의 코드는 수집 중 오류가 발생했을 때에만 실행해주세요!!

In [ ]:
min_len = min(len(postdate), len(titles), len(contents), len(urls_list))
if len(postdate) != len(contents):
        print(f"[경고] 리스트 길이 불일치: date={len(postdate)}, title={len(titles)}, content={len(contents)}, url={len(urls_list)}")


keyword_df = pd.DataFrame({
        'keyword': [keyword] * min_len,
        'date': postdate[:min_len],
        'title': titles[:min_len],
        'content': contents[:min_len],
        'url': urls_list[:min_len]
    })

all_data.append(keyword_df)

result_df = pd.concat(all_data, ignore_index=True)

In [ ]:
result_df

## 오류 미발생시 데이터 수집 완료 후 데이터 저장 코드부터 실행해주세요!

### 데이터 저장

In [ ]:
result_df = pd.concat(all_data, ignore_index=True)
result_df

In [ ]:
keywords

In [ ]:
name = '|'.join(keywords)
name

In [ ]:
result_df.to_csv(f"blog_crawling_result_{name}.csv", index=False, encoding='utf-8-sig')
print("✅ 전체 키워드 크롤링 완료 및 저장 완료")